In [9]:
from evaluate_output import MetaAnalysisTaskEvaluator
import utils
from collections import defaultdict
from typing import Dict
import yaml
import json

In [10]:
def categorize_outcome_type(output_file_path: str) -> Dict:
    """
    Categorizes the errors in the outcome type task
    Categories:
        1. Model outputs in an undesirable format
        2. Model outputs binary when its continuous
        3. Model outputs continuous when its binary
        4. Model outputs unknown when the reference is known
    
    Args:
        output_file_path: Output file path of the outcome type task
    
    Returns:
        Dict: A dictionary containing the error categories and the record ids that fall into each category
    """
    def is_badly_formatted_output(output: str) -> bool:
        return output not in ["A", "B", "C"]
    
    evaluator = MetaAnalysisTaskEvaluator('outcome_type', output_file_path, 'metrics/outcome_type/', None)
    evaluator.run_evaluation()
    
    errors = defaultdict(list)
    # character_to_string_mapping = {"A": "binary", "B": "continuous", "C": "x"}
    for record in evaluator.data:
        
        # Clean the output in the same way from `evaluate_output.py`
        output = record['output'].replace("The answer is ", "").replace(".", "").replace("(", "").replace(")","")
        for char in output:
            if not char.isspace():
                output = char
                break
               
        # Check for badly formatted outputs
        if is_badly_formatted_output(output):
            errors["badly_formatted_output"].append(record['id'])
            
        # Check for unknown when reference is known
        elif output == "C" and record['outcome_type'] != "":
            errors["unknown_when_reference_known"].append(record['id'])
        
        # Check for binary when continuous
        elif output == "B" and record['outcome_type'] == "binary":
            errors["binary_when_continuous"].append(record['id'])
            
        # Check for continuous when binary
        elif output == "A" and record['outcome_type'] == "continuous":
            errors["continuous_when_binary"].append(record['id'])
            
    return errors

def categorize_outcomes(output_file_path: str, outcome_type: str) -> Dict:
    """
    Categorizes the errors in the binary outcomes task
    Categories:
        1. Model outputs in an undesirable format
        2. Model has output but reference is unknown
        3. Model outputs unknown but reference is known
        4. Reference is and output is known but model outputs incorrect value
        
    Args:
        output_file_path: Output file path of the given task
        outcome_type: The outcome type of the task
        
    Returns:
        Dict: A dictionary containing the error categories and the record ids that fall into each category
    """
    evaluator = MetaAnalysisTaskEvaluator(outcome_type, output_file_path, 'metrics/' + outcome_type + '/', None)
    evaluator.run_evaluation()

    errors = defaultdict(list)
    
    # Define the reference keys and output keys based on the outcome type
    if outcome_type == 'binary_outcomes':
        reference_keys = ["intervention_events", "intervention_group_size", "comparator_events", "comparator_group_size"]
        output_keys = [f'{category}_output' for category in reference_keys]
    elif outcome_type == 'continuous_outcomes':
        reference_keys = ['intervention_mean', 'intervention_standard_deviation', 'intervention_group_size', 'comparator_mean', 'comparator_standard_deviation', 'comparator_group_size']
        output_keys = [f'{category}_output' for category in reference_keys]
    else:
        raise ValueError("Invalid outcome type")
    
    for record in evaluator.data:
        # Check for badly formatted outputs
        try:
            _ = yaml.safe_load(utils.clean_yaml_output(record['output']))
        except:
            errors["badly_formatted_output"].append(record['id'])
            continue
            
        # Check for output but reference is unknown
        for reference_key, output_key in zip(reference_keys, output_keys):
            if record[reference_key] == "x" and record[output_key] != "x":
                # Check if the record is already in the list
                if record['id'] not in errors["known_output_but_reference_unknown"]:
                    errors["known_output_but_reference_unknown"].append(record['id'])
                
            
            # Check for unknown output but reference is known
            if record[reference_key] != "x" and record[output_key] == "x":
                # Check if the record is already in the list
                if record['id'] not in errors["unknown_output_but_reference_known"]:
                    errors["unknown_output_but_reference_known"].append(record['id'])
                
                
            # Check for incorrect answer
            if record[reference_key] != "x" and record[output_key] != "x":
                if record[reference_key] != record[output_key]:
                    # Check if the record is already in the list
                    if record['id'] not in errors["incorrect_output"]:
                        errors["incorrect_output"].append(record['id'])
                        
    return errors

In [11]:
outcome_type_errors = categorize_outcome_type('outputs/outcome_type/olmo7B_outcome_type_test_output_20240326-12:06:30.json')

Metrics for the task:
{
    "number_of_model_unknowns": {
        "outcome_type": 5,
        "total": 5
    },
    "number_of_reference_unknowns": {
        "outcome_type": 0,
        "total": 0
    },
    "exact_match_accuracy": {
        "outcome_type": 0.2896341463414634,
        "total": 0.2896341463414634
    },
    "partial_match_accuracy": {
        "partial_match_accuracy_1": 0.2896341463414634
    },
    "outcome_type_f_score": {
        "outcome_type": {
            "f1_score_binary": 0.4239401496259351,
            "f1_score_continuous": 0.0792079207920792,
            "f1_score_unknown": 0.0
        }
    }
}


In [12]:
# print(json.dumps(outcome_type_errors, indent=4))
print("Outcome Type Errors:")
for key, value in outcome_type_errors.items():
    print(f"{key}: {len(value)}")

Outcome Type Errors:
continuous_when_binary: 461
unknown_when_reference_known: 5


In [13]:
binary_errors = categorize_outcomes('outputs/binary_outcomes/olmo7B_binary_outcomes_test_output_20240403-18:44:49.json', 'binary_outcomes')

Error in applying zero correction: Undefined results.
Error in applying zero correction: Undefined results.
Error in applying zero correction: Undefined results.
Error in applying zero correction: Undefined results.
Error in applying zero correction: Undefined results.
Error in applying zero correction: Undefined results.
Error in applying zero correction: Undefined results.
Error in applying zero correction: Undefined results.
Error in applying zero correction: Undefined results.
Error parsing yaml string: intervention:
    events: 10
    group_size: 397intervention:
    events: 787
    group_size: 392
Error parsing yaml string: intervention:
    events: 3
    group_size: 18intervention:
    events: 10
    group_size: 18
comparator:
    events: 6
    group_size: 18
Error in applying zero correction: Undefined results.
Error in applying zero correction: Undefined results.
Error parsing yaml string: intervention:
    events: 9
    group_size: 399intervention:
    events: 9
    group_siz

In [14]:
# print(json.dumps(binary_errors, indent=4))
print("Binary Outcome Errors:")
for key, value in binary_errors.items():
    print(f"{key}: {len(value)}")

Binary Outcome Errors:
unknown_output_but_reference_known: 14
badly_formatted_output: 115
incorrect_output: 44
known_output_but_reference_unknown: 5


In [15]:
continuous_errors = categorize_outcomes('outputs/continuous_outcomes/olmo7B_continuous_outcomes_test_output_20240403-21:36:24.json', 'continuous_outcomes')

An exception occurred for calculate standardized mean difference - intervention_mean: 0, control_mean: 0, intervention_sd: 0, control_sd: 0
An exception occurred for calculate standardized mean difference - intervention_mean: 80).2, control_mean: 75).2, intervention_sd: 7.77, control_sd: 7.41
Error parsing yaml string: intervention:
     mean: 120.0
     standard_deviation: 11.7
     group_size: 60
comparator:
     mean: 88.4
     standard_deviation: 11.8
    group_size: 60
Error parsing yaml string: intervention:
     mean: 4.0
     standard_deviation: 1.1
     group_size: 28
comparator:
     mean: 11.5
     standard_deviation: 3.3
    group_size: 28
Error parsing yaml string: intervention:
    mean: 83
    standard_deviation: 5
    group_size: 10
comparator:
    mean: 74
    standard_deviation: 5
    group_size: 10intervention:
    mean: 82
    standard_deviation: 5
    group_size: 10
An exception occurred for calculate standardized mean difference - intervention_mean: 80).0, control

In [16]:
# print(json.dumps(continuous_errors, indent=4))
print("Continuous Outcome Errors:")
for key, value in continuous_errors.items():
    print(f"{key}: {len(value)}")

Continuous Outcome Errors:
known_output_but_reference_unknown: 78
badly_formatted_output: 229
unknown_output_but_reference_known: 142
incorrect_output: 123
